# Modeling 1

## Libraries

In [5]:
# General libraries for data manipulation
import numpy as np
import pandas as pd

# Visualization libraries

import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing and sampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScarler
from imblearn.over_sampling import SMOTE





ImportError: cannot import name 'StandardScarler' from 'sklearn.preprocessing' (c:\Users\alexa\anaconda3\envs\customer_churn_Dec24\lib\site-packages\sklearn\preprocessing\__init__.py)

## Dataset

In [ ]:
df = pd.read_csv(r"C:\Users\alexa\OneDrive\AlexPerez\Dokumente\2 Soros\Data Science\Projects\customer_churn_Dec24\data\raw\Customer-Churn-Records.csv")
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis = 1)

display(df.head())
display(df.info())